<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/Python/04_statistical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. 識別 ー統計的手法ー

weatherデータでナイーブベイズ識別を行います。

In [1]:
import numpy as np
import pandas as pd
import io
from sklearn import preprocessing
from sklearn.naive_bayes import CategoricalNB

weatherデータをpandasのDataFrame形式で読み込みます。

In [2]:
weather_nominal = '''
outlook,temperature,humidity,windy,play
sunny,hot,high,FALSE,no
sunny,hot,high,TRUE,no
overcast,hot,high,FALSE,yes
rainy,mild,high,FALSE,yes
rainy,cool,normal,FALSE,yes
rainy,cool,normal,TRUE,no
overcast,cool,normal,TRUE,yes
sunny,mild,high,FALSE,no
sunny,cool,normal,FALSE,yes
rainy,mild,normal,FALSE,yes
sunny,mild,normal,TRUE,yes
overcast,mild,high,TRUE,yes
overcast,hot,normal,FALSE,yes
rainy,mild,high,TRUE,no
'''

In [3]:
df = pd.read_csv(io.StringIO(weather_nominal), sep=',')
print ("dataset: \n", df)

dataset: 
      outlook temperature humidity  windy play
0      sunny         hot     high  False   no
1      sunny         hot     high   True   no
2   overcast         hot     high  False  yes
3      rainy        mild     high  False  yes
4      rainy        cool   normal  False  yes
5      rainy        cool   normal   True   no
6   overcast        cool   normal   True  yes
7      sunny        mild     high  False   no
8      sunny        cool   normal  False  yes
9      rainy        mild   normal  False  yes
10     sunny        mild   normal   True  yes
11  overcast        mild     high   True  yes
12  overcast         hot   normal  False  yes
13     rainy        mild     high   True   no


scikit-learnのデータセット形式であるndarrayとして、パターン行列Xと正解yを取り出します。  

In [4]:
X = df.values[:,0:4]  #左から0列目から3列目まで
y = df.values[:,-1]   #右から1列目

特徴はOrdinalEncoderで数値ベクトルに変換します。
OrdinalEncoderはカテゴリ特徴を整数に置き換えます。

In [5]:
enc = preprocessing.OrdinalEncoder()
X_en = enc.fit_transform(X)

In [6]:
enc.categories_

[array(['overcast', 'rainy', 'sunny'], dtype=object),
 array(['cool', 'hot', 'mild'], dtype=object),
 array(['high', 'normal'], dtype=object),
 array([False, True], dtype=object)]

In [7]:
X_en

array([[2., 1., 0., 0.],
       [2., 1., 0., 1.],
       [0., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 1., 1.],
       [0., 0., 1., 1.],
       [2., 2., 0., 0.],
       [2., 0., 1., 0.],
       [1., 2., 1., 0.],
       [2., 2., 1., 1.],
       [0., 2., 0., 1.],
       [0., 1., 1., 0.],
       [1., 2., 0., 1.]])

正解は、LabelEncoderでyesとnoをそれぞれ1と0に置き換えます。

In [8]:
le = preprocessing.LabelEncoder()
y_en = le.fit_transform(y)

In [9]:
le.classes_

array(['no', 'yes'], dtype=object)

In [10]:
y_en

array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])

ナイーブベイズ識別器[CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html)の学習を行います。デフォルトでラプラス推定を行っています。

In [11]:
clf = CategoricalNB()
clf.fit(X_en, y_en)

CategoricalNB()

学習した統計モデルの中身を見ます。category\_count\_属性では、各特徴の情報が一つの行列で表されて、行方向がクラス、列方向が特徴の値です。データセット中の頻度が単純にカウントされているだけで、この属性にはスムージングの情報は入っていません。

In [16]:
clf.category_count_

[array([[0., 2., 3.],
        [4., 3., 2.]]),
 array([[1., 2., 2.],
        [3., 2., 4.]]),
 array([[4., 1.],
        [3., 6.]]),
 array([[2., 3.],
        [6., 3.]])]

スムージングされた結果は、feature\_log\_prob\_属性に入っています。

In [17]:
clf.feature_log_prob_

[array([[-2.07944154, -0.98082925, -0.69314718],
        [-0.87546874, -1.09861229, -1.38629436]]),
 array([[-1.38629436, -0.98082925, -0.98082925],
        [-1.09861229, -1.38629436, -0.87546874]]),
 array([[-0.33647224, -1.25276297],
        [-1.01160091, -0.45198512]]),
 array([[-0.84729786, -0.55961579],
        [-0.45198512, -1.01160091]])]

最初の行列から、クラスがyes（1行目）、outlook特徴の値がsunny（0列目）の値を取り出して、スムージングができていることを検算します。yesが9事例、そのうちsunnyが4事例なので、$\log\frac{4+1}{9+3}$を求めます。

In [18]:
import math
math.log(5/12)

-0.8754687373538999

例として (sunny, hot, high, FALSE) の判定結果を求めます。結果は、\[noの確率, yesの確率\]という形式のndarrayで帰ってきます。

In [22]:
clf.predict_proba(enc.transform([['sunny', 'hot', 'high', False]]))

array([[0.68796907, 0.31203093]])

教科書p.68の結果と少し数値が異なるのは、scikit-learnでは事前確率のスムージングを行っていないからです。下記計算で確認してください。

In [14]:
p=(3/12)*(3/12)*(4/11)*(7/11)*(9/14)
n=(4/8)*(3/8)*(5/7)*(3/7)*(5/14)
p/(p+n)

0.312030930179668

課題：スムージングを行わない方法でナイーブベイズ識別器の学習を行い、weatherデータ中の特定のデータに対する識別結果の確率を求め、その確率計算が正しいことを検算してください。

## 参考

* Naive Bayes https://scikit-learn.org/stable/modules/naive_bayes.html